In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
!pip install pymorphy2    #
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)
import re                 #
import csv                #
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

-----

# Задание 1

1. С помощью *pd.read_csv()* загрузите датафреймы positive.csv и negative.csv (обратите внимание, что исходные таблицы не содержат наименования столбцов и на первой строке располагаются данные. Файлы расположены в папке datasets);
2. Объедините датафреймы с помощью *pd.concat()* в один датафрейм;
3. Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
4. Переименуйте столбцы датафрейма (столбцы полностью соответствуют примеру);
5. Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

-----

---

## Семантический анализ твитов

Сегодня мы построим классификатор, который будет разделять текст на позитивные и негативные высказывания. Для этого мы воспользуемся уже размеченной базой.
Загрузим данные для анализа

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')   # чтобы можно было оттуда считать файл с данными для этого задания

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
col_names = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count']
df_pos = pd.read_csv('/content/drive/MyDrive/school21/day05/datasets/positive.csv', sep=';', names=col_names)
df_neg = pd.read_csv('/content/drive/MyDrive/school21/day05/datasets/negative.csv', sep=';', names=col_names)

In [ ]:
df_pos.shape

(114911, 12)

In [ ]:
df_neg.shape

(111923, 12)

In [ ]:
df = pd.concat([df_pos, df_neg], axis=0, ignore_index=True)

In [ ]:
df.shape

(226834, 12)

In [ ]:
len(df.id.unique())

226834

In [ ]:
len(df[df.id.duplicated() == True])

0

In [ ]:
df.positive.unique()

array([ 1, -1])

---

Все колонки таблицы могут содержать информацию о тональности твита, но мы будем ориентироваться исключительно на текст и на столбец отнесения к классу positiv.
Заменим значение -1 в колонке positive на 0

In [ ]:
df.positive[df.positive == -1] = 0

In [ ]:
len(df[df.id.duplicated() == True])

0

Количество строк в датафрейме соответствует количеству id, что также подтверждается нулевым значением количества дублирующих записей id. Таким образом, индексация сквозная и без повторов

Сохраним объединенный датафрейм в таблицу excel.

In [ ]:
nik_name = 'rriddler'
df.to_excel('/content/drive/MyDrive/school21/day05/datasets_'+nik_name+'.xlsx', index=False)

In [ ]:
print('Всего количество полученных твитов', len(df.positive), ', из них негативных ', len(df.positive[df.positive == 0]), ', позитивных ', len(df.positive[df.positive == 1]), '.') #Всего количество полученных твитов 226834 , из них негативных  111923 , позитивных  114911 .

Всего количество полученных твитов 226834 , из них негативных  111923 , позитивных  114911 .


----

# Задание 2

Произведите очистку данных, сформированных в задании 1. По результатам очистки выведите на экран следующую информацию:   
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.

----

***Совет:*** сохраняйте промежуточные результаты очистки, чтобы в случае неверных действий на каком-либо этапе не пересчитывать все предыдущие этапы

---

### Очистка и предобработка данных

Перед разработкой классификатора нам необходимо очистить и предобработать данные.

Начнем с очистки данных

----------------------------

***Внимание!*** Библиотека [*nltk*](https://www.nltk.org) может содержать не все компоненты. В случае возникновения ошибки необходимо запустить скрипт

*import nltk   
nltk.download()*

В открывшемся окне необходимо выбрать и установить требуемые компоненты

----------------------------

Приведем весь текст к строчным буквам:

In [ ]:
df.text = df.text.str.lower()
df.text.loc[19:22]

19    @moscow_advokat очень главное спасибо for   ме...
20    у нас есть прекрасная история, как сдохнуть за...
21    @benjamin1610 сегодня столько поводов выпить, ...
22    @stalingulag унылый?...наверное...вдруг его пе...
Name: text, dtype: object

---

Оставим в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницей:

In [ ]:
df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[19:22]

19                    очень главное спасибо       ме...
20    у нас есть прекрасная история  как сдохнуть за...
21                  сегодня столько поводов выпить  ...
22                 унылый    наверное   вдруг его пе...
Name: text, dtype: object

---

Мы анализируем русскоязычный твиттер, поэтому английские слова, а так же числа, будут представлять частные случаи и формировать шум в данных. Но могут возникнуть задачи, где удаляемые слова и числа важны. В этом случае потребуется более взвешенный подход к очистке. Вам могут помочь [константы модуля *string*](https://docs.python.org/3/library/string.html)

Разобьем тексты на слова с помощью *word_tokenize*:

In [ ]:
from nltk.tokenize import word_tokenize
df.text = list(map(word_tokenize, df.text))
df.text.loc[19:22]

19    [очень, главное, спасибо, медвед, он, работал,...
20    [у, нас, есть, прекрасная, история, как, сдохн...
21    [сегодня, столько, поводов, выпить, что, я, уж...
22    [унылый, наверное, вдруг, его, перед, этим, чп...
Name: text, dtype: object

---

В каждом языке имеются так называемые стоп-слова - это, например, предлоги, союзы, местоимения и т.д. Стоп-слова не несут смысловой нагрузки, но при этом встречаются достаточно часто. Существует множество словарей стоп-слов, мы воспользуемся словарем библиотеки *nltk*. При решении конкретных задач вы можете как расширить словарь стоп-слов, так и удалить из него любые слова.

In [ ]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

---

Удалим стоп-слова из наших данных

In [ ]:
def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[19:22]

19    [очень, главное, спасибо, медвед, работал, кло...
20              [прекрасная, история, сдохнуть, неделю]
21    [сегодня, столько, поводов, выпить, могу, усид...
22                   [унылый, наверное, этим, чпокнули]
Name: text, dtype: object

---

Проведем [лемматизацию](https://ru.wikipedia.org/wiki/Лемматизация) полученных слов

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[19:22]

19    [очень, главное, спасибо, медведа, работать, к...
20              [прекрасный, история, сдохнуть, неделя]
21    [сегодня, столько, повод, выпить, мочь, усидет...
22                    [унылый, наверное, это, чпокнуль]
Name: text, dtype: object

In [ ]:
df.to_excel('/content/drive/MyDrive/school21/day05/datasets_lemm_'+nik_name+'.xlsx', index=False)

---

Теперь необходимо удалить все слова, которые встречаются только 1 раз

In [ ]:
from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))
words_to_del = set(words_to_del)

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

df.text = list(map(delete_word, df.text))
df.text = list(map(to_str, df.text))
df.text.loc[19:22]

19    очень главное спасибо медведа работать клоун а...
20                   прекрасный история сдохнуть неделя
21      сегодня столько повод выпить мочь усидеть место
22                                  унылый наверное это
Name: text, dtype: object

In [ ]:
df.to_excel('/content/drive/MyDrive/school21/day05/datasets_del_'+nik_name+'.xlsx', index=False)

---

На этом очистка данных завершена. Можно ли утверждать, что очистка идеальна? Однозначно нет! Но, ее может оказаться достаточно для решения нашей задачи.   
Какие еще задачи могут возникнуть при очистке текстовых данных? Вот далеко неполный список:
- Обработка больших документов и больших коллекций текстовых документов, которые не помещаются в память.
- Извлечение текста из разметки, такой как HTML, PDF или другие структурированные форматы документов.
- Транслитерация символов с других языков.
- Декодирование символов Юникода в нормализованную форму, такую как UTF8
- Обработка доменных имен, фраз и сокращений.
- Обработка или удаление чисел, таких как даты и суммы.
- Поиск и исправление распространенных опечаток и ошибок в написании.

Можно очень долго заниматься очисткой и не достичь идеального результата. Лучше подойти к задаче итеративно - осуществить стандартную очистку и посмотреть на результат, если результат недостаточный, то провести дополнительные мероприятия по очистке.

In [ ]:
print(len(text_tokens), '- общее количество слов перед удалением слов, встречающихся 1 раз;', len(words_to_del), '– количество слов, встречающихся 1 раз;',  len(text_tokens) - len(words_to_del), '– итоговое количество слов после очистки.') #(1493275, 53568, 1439707)

1493275 - общее количество слов перед удалением слов, встречающихся 1 раз; 53568 – количество слов, встречающихся 1 раз; 1439707 – итоговое количество слов после очистки.


---
После очистки могут оказаться пустые твиты, т.е. эти твиты состояли из слов, записанных латиницей, стоп-слов, чисел, знаков припинания и уникальных слов. Такие твиты необходимо удалить из данных:

In [ ]:
print(len(df[df.text == '']), '– количество пустых твитов', len(df[(df.text == '') & (df.positive == 1)]), '– из них позитивных твитов.') #(856, 382)

856 – количество пустых твитов 382 – из них позитивных твитов.


In [ ]:
df = df.drop(df[df.text == ''].index, axis = 0) #225978
print(len(df), '– количество твитов после удаления пустых')

225978 – количество твитов после удаления пустых


---

# Задание 3

1. Кодировать данные методом мешка слов.
2. Кодировать данные методом TF-IDF.
3. Построить классификатор на основе логистической регрессии, используя мешок слов.
4. Построить классификатор на основе логистической регрессии, используя TF-IDF.
5. Построить классификатор на основе случайного леса, используя мешок слов.
6. Построить классификатор на основе случайного леса, используя TF-IDF.
7. Сделайте выводы о разработанных классификаторах.

---

При разбиении на обучающую и тестовую выборки, следует указать *test_size=0.3*

---

***Рекомендация:*** для случайного леса параметр n_estimator должен быть не менее 200

Для разработки моделей нам необходимо оцифровать полученные данные. Мы воспользуемся двумя методами: мешком слов [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и TF-IDF [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Но в начале необходимо разбить данные на обучающую и тестовую выборки.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.3, random_state=21)

---

Рассмотрим количество твитов в выборке для обучения, из них позитивных, и в выборке для теста, из них позитивных

In [ ]:
len(y_train), y_train.sum(), len(y_test), y_test.sum()

(158184, 80157, 67794, 34372)

---

### Кодировка данных   

Кодируем наши данные мешком слов и tf-idf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

---

### Классификаторы

Построим классификатор с помощью логистической регрессии:   
на основе мешка слов:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.72      0.71      0.71     33422
           1       0.72      0.73      0.72     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.80      0.79      0.79     78027
           1       0.80      0.80      0.80     80157

    accuracy                           0.80    158184
   macro avg       0.80      0.80      0.80    158184
weighted avg       0.80      0.80      0.80    158184



---

на основе tf-idf:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       0.73      0.69      0.71     33422
           1       0.71      0.75      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       0.79      0.75      0.77     78027
           1       0.77      0.80      0.79     80157

    accuracy                           0.78    158184
   macro avg       0.78      0.78      0.78    158184
weighted avg       0.78      0.78      0.78    158184



---

Видно, что модели переобучены - это следствие малого количества данных.

---

Построим классификатор с помощью случайного леса [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html):   
на основе мешка слов:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.68      0.76      0.72     33422
           1       0.74      0.65      0.69     34372

    accuracy                           0.70     67794
   macro avg       0.71      0.71      0.70     67794
weighted avg       0.71      0.70      0.70     67794

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



---

на основе tf-idf:

In [ ]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       0.72      0.70      0.71     33422
           1       0.72      0.74      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



#Выводы: 
----
Классификатор на основе логистической регрессии при использовании tf-idf на тестовой выборке показал лучший f1-score, чем при использовании мешка слов.
----
Классификатор на основе случайного леса, используя мешок слов, показал больше значение f1-score, чем при использовании TF-IDF.
Модель на RandomForest возможно переобучена, она на обучающей выборке показала f1-score 1, что значительно выше метрик по результатам обучения логистической регрессией. Снижение гиперпараметра n_estimators с 200 на 100 не влияет на результат, снижение до 3 плавно ухудшает f1 с 0.7 до 0. 